In [ ]:
!unzip ../input/A榜/toUserA.zip -d ../output/A榜/

In [ ]:
!pip install sklearn
!pip install tdqm
!pip install pandas==1.3.0
!pip install lightgbm

In [ ]:
import gc
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tqdm 
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

In [ ]:
col = 'userid'
print(col)
df_train = pd.read_csv('../output/toUserA/train.csv', usecols=[col])
df_test = pd.read_csv('../output/toUserA/testa_nolabel.csv', usecols=[col])
print('LE ...')
le = LabelEncoder()
le.fit(pd.concat([df_train[col], df_test[col]]))
df_train[col] = le.transform(df_train[col])
df_test[col] = le.transform(df_test[col])
df_train[col] = df_train[col].astype('uint16')
df_test[col] = df_test[col].astype('uint16')
df_train.to_pickle('../output/train_'+col+'.pkl')
del df_train
df_test.to_pickle('../output/test_'+col+'.pkl')
del df_test, le
gc.collect()

In [ ]:
col = 'videoid'
print(col)
df_train = pd.read_csv('../output/toUserA/train.csv', usecols=[col], chunksize=10000000)
df_test = pd.read_csv('../output/toUserA/testa_nolabel.csv', usecols=[col])
i = 0
new_train = pd.DataFrame()
for chunk in df_train:
    print(i, new_train.shape)
    i += 1
    chunk[col] = chunk[col].map(lambda x: sum([(ord(x[i])-48) * 36**i for i in range(len(x))]))
    new_train = pd.concat([new_train, chunk])
df_train = new_train
df_test[col] = df_test[col].map(lambda x: sum([(ord(x[i])-48) * 36**i for i in range(len(x))]))
print('LE ...')
le = LabelEncoder()

# le.fit(pd.concat([df_train[col], df_test[col]]))
df_train[col] = le.fit_transform(df_train[col])
df_test[col] = le.transform(df_test[col])
df_train[col] = df_train[col].astype('uint32')
df_test[col] = df_test[col].astype('uint32')
df_train.to_pickle('../output/train_'+col+'.pkl')
del df_train
df_test.to_pickle('../output/test_'+col+'.pkl')
del df_test, le
gc.collect()

In [ ]:
col = 'tag'
print(col)
df_train = pd.read_csv('../output/toUserA/train.csv', usecols=[col])
df_test = pd.read_csv('../output/toUserA/testa_nolabel.csv', usecols=[col])
print('LE ...')
le = LabelEncoder()
le.fit(pd.concat([df_train[col], df_test[col]]))
df_train[col] = le.transform(df_train[col])
df_test[col] = le.transform(df_test[col])
df_train[col] = df_train[col].astype('uint8')
df_test[col] = df_test[col].astype('uint8')
df_train.to_pickle('../output/train_'+col+'.pkl')
del df_train
df_test.to_pickle('../output/test_'+col+'.pkl')
del df_test, le
gc.collect()

In [ ]:
gc.collect()

In [ ]:
df_train = pd.DataFrame()
for col in ['userid', 'videoid', 'tag']:
    print(col)
    df_train = pd.concat([df_train, pd.read_pickle('../output/train_'+col+'.pkl')], axis=1)
others = pd.read_csv('../output/toUserA/train.csv', 
                     usecols=['is_finish'],
                     dtype={'is_finish': bool})
df_train = pd.concat([df_train, others], axis=1)
del others

df_test = pd.DataFrame()
for col in ['userid', 'videoid', 'tag']:
    print(col)
    df_test = pd.concat([df_test, pd.read_pickle('../output/test_'+col+'.pkl')], axis=1)

df_train.to_pickle('../output/df_train.pkl')
df_test.to_pickle('../output/df_test.pkl')

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2222)
for fold, (train_idx, val_idx) in enumerate(kf.split(df_train.index, df_train['is_finish'])):
    break
df_train = df_train.loc[val_idx].reset_index(drop=True)

In [ ]:
df_train.to_pickle('../output/df_train_subset.pkl')

In [ ]:
# df_train = pd.read_pickle('../output/df_train_subset.pkl')
# df_test = pd.read_pickle('../output/df_test.pkl')

In [ ]:
df_train.shape, df_test.shape

In [ ]:
LABEL = 'is_finish'
feats = [f for f in df_test.columns]
print(feats)
params = {
    'learning_rate': 0.5,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'verbose': -1,
    'seed': 2222,
    'n_jobs': 4,
}

fold_num = 5
seed = 2222
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
for fold, (train_idx, val_idx) in enumerate(kf.split(df_train.index, df_train[LABEL])):
    print('-----------', fold)
    train = lgb.Dataset(df_train.loc[train_idx, feats],
                        df_train.loc[train_idx, LABEL])
    val = lgb.Dataset(df_train.loc[val_idx, feats],
                      df_train.loc[val_idx, LABEL])
    print('Train')
    model = lgb.train(params, train, valid_sets=[val], num_boost_round=10000, 
                      callbacks=[lgb.early_stopping(50), lgb.log_evaluation(50)])
    oof[val_idx] += model.predict(df_train.loc[val_idx, feats])
    pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
    importance += model.feature_importance(importance_type='gain') / fold_num
    break

feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
print(feats_importance.sort_values('importance', ascending=False)[:30])

df_train['oof'] = oof
# score = log_loss(df_train[LABEL], df_train['oof'])
score = log_loss(df_train.loc[val_idx, LABEL], df_train.loc[val_idx, 'oof'])

print(score)
# df_test[LABEL] = pred_y.mean(axis=1).values

In [ ]:
submit = pd.read_csv('../output/toUserA/submita.csv')
submit['is_finish'] = pred_y.mean(axis=1)
submit.to_csv('../output/ans/lgb_baseline.csv', index=False)

In [ ]:
!castlecli --third codeon --token xxxxxxx --source ../output/ans/lgb_baseline.csv